In [12]:
from datetime import datetime
import pandas as pd
import numpy as np
from functools import reduce




## Preprocessing and feature extraction functions

### Time preprocessing 

In [13]:


#  VisitDateTime contains unixtime and datetime as string type, so parsing and formating their values
def time_preprocessing(visitorlog):
    def tme(x):
        if x is not np.nan: 
            try: # First condition for converting unixtime to datetime
                
                x = int(x)/(10**9) # unixtime has been suffixed with 10^9, so removing it   
                x = datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S.%f')
                
            except: # second condition for converting date string to datetime
                x = datetime.strptime(str(x),'%Y-%m-%d %H:%M:%S.%f')


        return(x)
    
    # Using apply function for implementing tme function to each row
    visitorlog['VisitDateTime']=visitorlog['VisitDateTime'].apply(lambda x: tme(x))
    return visitorlog











### No_of_days_Visited_7_Days


In [14]:
def No_of_days_Visited_7_Days_f(vistorlog_mappedv1) :
    
    #We will use filter in 'day' column for generating a flag of user activeness in website for 7 days window
    vistorlog_mappedv1['active7'] = np.where(((vistorlog_mappedv1['day']>14.0) & (vistorlog_mappedv1['day']<22.0)),1,0) 
    
    # Grouping by Users which were active in between of those 7 days
    active7_days =vistorlog_mappedv1[vistorlog_mappedv1['active7']==1].loc[:,['UserID','day']].groupby(['UserID']).day.nunique()
    active7_days = pd.DataFrame(active7_days.reset_index())
    active7_days.rename({'day':'No_of_days_Visited_7_Days'},axis = 1,inplace = True)
    
    # Joining with User table
    active7_days = pd.merge(pd.DataFrame(user['UserID']),active7_days,how = 'left')
    # Filling NA with 0
    active7_days.fillna(0,inplace = True)
    
    return active7_days,vistorlog_mappedv1


### No_Of_Products_Viewed_15_Days 

In [15]:
def No_Of_Products_Viewed_15_Days_f(vistorlog_mappedv1):
    
    # creating a flag column for non null product ids with day window range of 15 days 
    vistorlog_mappedv1['product15'] = np.where(((vistorlog_mappedv1['day']>6.0) & (vistorlog_mappedv1['day']<22.0) & (vistorlog_mappedv1['ProductID'].notna())),1,0)
   
    # ProductID have upper and lower case, so conveting all to lower case
    vistorlog_mappedv1['ProductID']= vistorlog_mappedv1['ProductID'].str.lower()
    
    # Getting count of unique product id with respect to user id
    product15_days =vistorlog_mappedv1[vistorlog_mappedv1['product15']==1].loc[:,['UserID','ProductID']].groupby(['UserID']).ProductID.nunique()
    product15_days = pd.DataFrame(product15_days.reset_index())
    product15_days.rename({'ProductID':'No_Of_Products_Viewed_15_Days'},axis = 1,inplace = True)
    
    # Joining with User table
    product15_days = pd.merge(pd.DataFrame(user['UserID']),product15_days,how = 'left')
    
    product15_days.fillna(0,inplace = True)
    
    # Filling NA with 0
    return product15_days,vistorlog_mappedv1

### User_Vintage

In [16]:
def User_Vintage_f(user):
    user_vintage =user.loc[:,['UserID','Signup Date']]
    # Parsing string date to datetime type, x[:10] signifies the inclusion of only date part
    user_vintage['Signup Date']= user_vintage['Signup Date'].apply(lambda x: datetime.strptime(x[:10],'%Y-%m-%d'))
    
    # According to problem statement, today date is 28th May,2018
    user_vintage['date_now'] = datetime(2018,5,28,0,0,0)
    
    # Difference between today and User signup date is User_Vintage
    user_vintage['User_Vintage'] = user_vintage['date_now'] -  user_vintage['Signup Date']
    user_vintage['User_Vintage'] = user_vintage['User_Vintage'].dt.days
    user_vintage = user_vintage.loc[:,['UserID','User_Vintage']]
    
    return user_vintage



### Most_Viewed_product_15_Days

In [17]:
def Most_Viewed_product_15_Days_f(vistorlog_mappedv1):
    
    # Converting all Activity to lowercase
    vistorlog_mappedv1['Activity']= vistorlog_mappedv1['Activity'].str.lower()
    
    # Creating flag for Users which pageloaded product in between 15 days window
    vistorlog_mappedv1['product15_pageload'] = np.where(((vistorlog_mappedv1['day']>6.0) & (vistorlog_mappedv1['day']<22.0) & (vistorlog_mappedv1['ProductID'].notna()) & (vistorlog_mappedv1['Activity']=='pageload')),1,0)
    
    # Getting count of each product viewed by UserID
    product15_pageload = vistorlog_mappedv1[vistorlog_mappedv1['product15_pageload']==1].groupby(['UserID','ProductID']).size()

    product15_pageload = pd.DataFrame(product15_pageload.reset_index())
    product15_pageload.rename({0:'Products_pageload_count'},axis = 1,inplace = True)
    
    # Getting most viewed productid 
    idx = product15_pageload.groupby('UserID').Products_pageload_count.transform(max) == product15_pageload['Products_pageload_count']
    product15_pageload=product15_pageload[idx]
    
    # Creating Another dataframe where specifying maximum time for each product id
    
    product15_pageload_time = vistorlog_mappedv1[vistorlog_mappedv1['product15_pageload']==1].groupby(['UserID','ProductID']).VisitDateTime.max()      
    product15_pageload_time = pd.DataFrame(product15_pageload_time.reset_index())
    product15_pageload_time.rename({0:'max_time'},axis = 1,inplace = True)
    
    # Joining product15_pageload_time (max-time containing dataframe) with product15_pageload
    product15_pageload_final = pd.merge(product15_pageload,product15_pageload_time,on = ['UserID','ProductID'],how = 'left')
    
    # If multiple products that have a similar number of page loads then it will consider most recent time
    idx = product15_pageload_final.groupby('UserID').VisitDateTime.transform(max) == product15_pageload_final['VisitDateTime']
    product15_pageload_final=product15_pageload_final[idx]
    
    
    # Some products were viewed at same time, so considering one of those products
    product15_pageload_finalv1 = product15_pageload_final.loc[:,['UserID','ProductID']]
    df = product15_pageload_finalv1.groupby('UserID').ProductID.transform(max) == product15_pageload_final['ProductID']
    product15_pageload_finalv1= product15_pageload_finalv1[df]
    
    
    # Some Userid haven't viewed any product so filling NA with product101
    Most_Viewed_product_15_Days = pd.merge(pd.DataFrame(user['UserID']),product15_pageload_finalv1,on = 'UserID',how= 'left')
    Most_Viewed_product_15_Days.fillna('product101',inplace =True)
    
    Most_Viewed_product_15_Days.rename({'ProductID':'Most_Viewed_product_15_Days'},axis = 1,inplace = True)

    return Most_Viewed_product_15_Days,vistorlog_mappedv1

### Most_Active_OS

In [18]:
def Most_Active_OS_f(vistorlog_mappedv1):
    # Converting OS values to lowercase
    vistorlog_mappedv1['OS'] = vistorlog_mappedv1['OS'].str.lower()
    # Count of each OS with respect to Userid 
    Most_Active_OS= vistorlog_mappedv1.groupby(['UserID','OS']).size()
    Most_Active_OS= pd.DataFrame(Most_Active_OS.reset_index())
    Most_Active_OS.rename({0:'Most_Active_OS'},axis = 1,inplace = True)
    # Considering OS with maximum count 
    idx = Most_Active_OS.groupby('UserID').Most_Active_OS.transform(max) == Most_Active_OS['Most_Active_OS']
    Most_Active_OS= Most_Active_OS[idx]
    Most_Active_OS = Most_Active_OS.loc[:,['UserID','OS']]
    
    # Users have same counts for both OS
    df = Most_Active_OS.groupby('UserID').OS.transform(max) == Most_Active_OS['OS']
    Most_Active_OS=Most_Active_OS[df]
    
    Most_Active_OS.rename({'OS':'Most_Active_OS'},axis = 1,inplace = True)
    return Most_Active_OS,vistorlog_mappedv1



### Recently_Viewed_Product

In [19]:
def Recently_Viewed_Product_f(vistorlog_mappedv1):

    # Filtering pageload activity
    Recently_Viewed_Product  = vistorlog_mappedv1[vistorlog_mappedv1['Activity']=='pageload']
    Recently_Viewed_Product = Recently_Viewed_Product.loc[:,['UserID','ProductID','VisitDateTime']]
    # Getting most recent product view by user
    idx = Recently_Viewed_Product.groupby('UserID').VisitDateTime.transform(max) == Recently_Viewed_Product['VisitDateTime']
    Recently_Viewed_Product = Recently_Viewed_Product[idx]
    
    # Some products have same visit time, so considering first
    Recently_Viewed_Product = Recently_Viewed_Product.loc[:,['UserID','ProductID']]
    Recently_Viewed_Product=Recently_Viewed_Product.groupby('UserID').ProductID.first()
    Recently_Viewed_Product =pd.DataFrame(Recently_Viewed_Product.reset_index())
    
    # Filling NAs
    Recently_Viewed_Product = pd.merge(pd.DataFrame(user['UserID']),Recently_Viewed_Product,on = 'UserID',how= 'left')
    Recently_Viewed_Product.fillna('product101',inplace =True)
    
    Recently_Viewed_Product.rename({'ProductID':'Recently_Viewed_Product'},axis = 1,inplace = True)
    
    return Recently_Viewed_Product,vistorlog_mappedv1

### Pageloads_last_7_days 

In [20]:
def  Pageloads_last_7_days_f(vistorlog_mappedv1,user):
    # Using previous generated 7 day flag 
    Pageloads_last_7_days = vistorlog_mappedv1[(vistorlog_mappedv1['active7']==1) & (vistorlog_mappedv1['Activity']=='pageload')].loc[:,['UserID','Activity']]
    
    # Getting count of pageload by userid
    Pageloads_last_7_days = Pageloads_last_7_days.groupby('UserID').Activity.count()
    Pageloads_last_7_days = pd.DataFrame(Pageloads_last_7_days.reset_index())
    Pageloads_last_7_days.rename({'Activity':'Pageloads_last_7_days'},axis = 1, inplace = True)
    
    # merging with user table and filling na
    Pageloads_last_7_days = pd.merge(pd.DataFrame(user['UserID']),Pageloads_last_7_days,on = 'UserID',how= 'left')
    Pageloads_last_7_days.fillna(0,inplace =True)
    
    return Pageloads_last_7_days

    

### Clicks_last_7_days

In [21]:
def Clicks_last_7_days_f(vistorlog_mappedv1,user):
    # Using previous generated 7 day flag               
    Clicks_last_7_days= vistorlog_mappedv1[(vistorlog_mappedv1['active7']==1) & (vistorlog_mappedv1['Activity']=='click')].loc[:,['UserID','Activity']]
    
    # Getting count of pageload by userid                   
    Clicks_last_7_days= Clicks_last_7_days.groupby('UserID').Activity.count()
    Clicks_last_7_days= pd.DataFrame(Clicks_last_7_days.reset_index())
    Clicks_last_7_days.rename({'Activity':'Clicks_last_7_days'},axis = 1, inplace = True)
    
    # merging with user table and filling na
    Clicks_last_7_days= pd.merge(pd.DataFrame(user['UserID']),Clicks_last_7_days,on = 'UserID',how= 'left')
    Clicks_last_7_days.fillna(0,inplace =True)
    return Clicks_last_7_days

## Main Code


In [22]:



# Loading visitorlog data and User data
visitorlog = pd.read_csv('data/VisitorLogsData.csv')
user = pd.read_csv('data/userTable.csv')

# Considering data where UserID is available
visitorlog=visitorlog[visitorlog['UserID'].notna()]
print(visitorlog.shape)


# calling time preprocessing function
visitorlog = time_preprocessing(visitorlog)
visitorlog.info()

# We have 21 days window, so creating a feature 'day' which will get the day number ranging from 1 to 21
min_day = visitorlog['VisitDateTime'].min().day - 1
visitorlog['day']=visitorlog['VisitDateTime'].apply(lambda x: x.day - min_day)

# Joining visitorlog and user data
vistorlog_mapped = pd.merge(user,visitorlog,on = 'UserID',suffixes=('_left',''))

# Treating NA time values in Visitorlog data

na_date = vistorlog_mapped[vistorlog_mapped['day'].isna()] # records with NA day values
not_na_date = vistorlog_mapped[vistorlog_mapped['day'].notna()] # Records without non null day values

# Imputing missing 'VisitDateTime' and 'day' values with minimum datetime values respective to webClientID

group_clientid = not_na_date[['webClientID','VisitDateTime','day']].groupby('webClientID').min()
date_na_treated = pd.merge(group_clientid,na_date,how='right',on = 'webClientID',suffixes=('','_right'))

# 'VisitDateTime_right' and 'day_right' are from na_date dataframe, so they have all null values
# We will drop them
date_na_treated.drop(['VisitDateTime_right','day_right'],axis = 1,inplace = True)

# Appending date_na_treated and not_na_date

# Alligning column order of date_na_treated with not_na_date before append
date_na_treated = date_na_treated[['UserID', 'Signup Date', 'User Segment', 'webClientID', 'VisitDateTime',
       'ProductID', 'Activity', 'Browser', 'OS', 'City', 'Country', 'day']]
#append
vistorlog_mappedv1 =not_na_date.append(date_na_treated)

# Calling function No_of_days_Visited_7_Days

active7_days,vistorlog_mappedv1 = No_of_days_Visited_7_Days_f(vistorlog_mappedv1)

# Calling No_Of_Products_Viewed_15_Days

product15_days,vistorlog_mappedv1 = No_Of_Products_Viewed_15_Days_f(vistorlog_mappedv1)

# Calling User_Vintage
user_vintage = User_Vintage_f(user)

# Calling Most_Viewed_product_15_Days

Most_Viewed_product_15_Days,vistorlog_mappedv1 = Most_Viewed_product_15_Days_f(vistorlog_mappedv1)

# Calling Most_Active_OS
Most_Active_OS,vistorlog_mappedv1 = Most_Active_OS_f(vistorlog_mappedv1)

# Calling Recently_Viewed_Product
Recently_Viewed_Product,vistorlog_mappedv1 = Recently_Viewed_Product_f(vistorlog_mappedv1)

# calling Pageloads_last_7_days
Pageloads_last_7_days = Pageloads_last_7_days_f(vistorlog_mappedv1,user)

# calling Clicks_last_7_days
Clicks_last_7_days = Clicks_last_7_days_f(vistorlog_mappedv1,user)


# Merging all features

df = [active7_days,product15_days,
       user_vintage,Most_Viewed_product_15_Days, Most_Active_OS,
       Recently_Viewed_Product, Pageloads_last_7_days,
       Clicks_last_7_days]
df_merged = reduce(lambda left,right: pd.merge(left,right,on = 'UserID',how = 'inner'),df)

# Sorting values by userid

df_merged.sort_values('UserID',inplace = True)

df_merged.reset_index(inplace = True)
df_merged = df_merged.iloc[:,1:]
df_merged.head()




(650695, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 650695 entries, 14 to 6587994
Data columns (total 9 columns):
webClientID      650695 non-null object
VisitDateTime    585892 non-null datetime64[ns]
ProductID        598732 non-null object
UserID           650695 non-null object
Activity         562938 non-null object
Browser          650695 non-null object
OS               650695 non-null object
City             414571 non-null object
Country          607868 non-null object
dtypes: datetime64[ns](1), object(8)
memory usage: 49.6+ MB


,UserID,No_of_days_Visited_7_Days,No_Of_Products_Viewed_15_Days,User_Vintage,Most_Viewed_product_15_Days,Most_Active_OS,Recently_Viewed_Product,Pageloads_last_7_days,Clicks_last_7_days
0,U100002,0.0,2.0,53,pr100258,android,pr100258,0.0,0.0
1,U100003,1.0,2.0,1021,pr100079,windows,pr100079,1.0,2.0
2,U100004,1.0,15.0,341,pr100753,windows,product101,1.0,0.0
3,U100005,1.0,3.0,681,pr100234,android,pr100234,1.0,0.0
4,U100006,1.0,1.0,55,pr101111,android,pr101111,1.0,0.0


In [23]:
df_merged.to_csv('submission1.csv')